In [1]:
from scipy.stats import mode
from numpy import linalg as LA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numba
import time
from scipy.sparse.linalg import eigs
from numpy.linalg import inv
import sys
from PFSA import PFSA

<hr style="border:2px solid blue"> </hr>

## load the in-core and ex-core detector signal

In [2]:
# import the in-core and ex-core detector signal
Xn_IC = np.load("X_incore_normal.npz")
Xa_IC = np.load("X_incore_abnormal.npz")

Xn_EC = np.load("X_excore_normal.npz")
Xa_EC =  np.load("X_excore_abnormal.npz")

print(Xn_IC.shape)
print(Xa_IC.shape)
print(Xn_EC.shape)
print(Xa_EC.shape)

(1000000001,)
(1000000001,)
(1000000001,)
(1000000001,)


<hr style="border:2px solid blue"> </hr>

## investigate the effect of alphabet size on the accuracy of the PFSA for neutron noise anomaly detection

In [3]:
alphabet_sizes = [5, 10, 20, 40, 50, 60, 70, 80, 100, 150, 200, 400, 800] # alphabet sizes

### part 1: for  in-core signal

In [ ]:
acc = []
for alphabet_size in alphabet_sizes:
    
    X_class1 = Xn_IC[1:] # class 1 is normal, index is 0
    X_class2 = Xa_IC[1:] # class 2 is abnormal, index is 1
    
    fs = 1000
    Ns = 1000000000
    
    window = int(fs * 100) # 100 seconds
    sample_size = len(X_class1) // window
    X_class1 = X_class1.reshape(sample_size, window)
    X_class1 = X_class1.reshape(*X_class1.shape, 1)
    np.random.shuffle(X_class1)
    Y_class1 = np.zeros(sample_size) # class 1 is labeled by index 0

    X_class2 = X_class2.reshape(sample_size, window)
    X_class2 = X_class2.reshape(*X_class2.shape, 1)
    np.random.shuffle(X_class2)
    Y_class2 = np.ones(sample_size) # class 2 is labled by index 1

    X_train = np.zeros((X_class1.shape[0], X_class1.shape[1], X_class1.shape[2]))
    Y_train = np.zeros(Y_class1.size)

    X_train[0: sample_size // 2] = X_class1[0: sample_size // 2].copy()
    X_train[sample_size // 2 : ] = X_class2[0: sample_size // 2].copy()

    Y_train[0: sample_size // 2] = Y_class1[0: sample_size // 2].copy()
    Y_train[sample_size // 2 : ] = Y_class2[0: sample_size // 2].copy()

    X_test = np.zeros((X_class1.shape[0], X_class1.shape[1], X_class1.shape[2]))
    Y_test = np.zeros(Y_class1.size)

    X_test[0: sample_size // 2] = X_class1[sample_size // 2:].copy()
    X_test[sample_size // 2 : ] = X_class2[sample_size // 2:].copy()

    Y_test[0: sample_size // 2] = Y_class1[sample_size // 2:].copy()
    Y_test[sample_size // 2 : ] = Y_class2[sample_size // 2:].copy()
    
    classfiers = ['projection 2'] # select to only use projection 2 classifier

    n_partitioning = alphabet_size
    depth = 1

    print("fs:", fs, "Ns:", Ns, "window:", window, "n_partitioning", n_partitioning)
    normalizaiton_method = 'mixed classes' # select to use "mixed classes" normalization method
    model = PFSA(n_partitioning = n_partitioning, depth = depth, train_regime= normalizaiton_method)
    model.fit(X_train, Y_train)
    y_test_predicted_raw  = model.predict(X_test, classifier=classfiers)
    y_test_predicted = y_test_predicted_raw
    acc.append(sum(y_test_predicted  == Y_test) / len(Y_test))
    print(acc)
    np.save("acc_incore_alphabet_sizes.npy", acc)  # save during running

fs: 1000 Ns: 1000000000 window: 100000 n_partitioning 5
normalization for all classes together
get the same bounds for all classes


/home/xxc90/neutronNoise_paper/PFSA.py:451: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "determine_partitioning_bounds" failed type inference due to: No implementation of function Function(<built-in function mul>) found for signature:
 
 >>> mul(int64, list(int64)<iv=None>)
 
There are 12 candidate implementations:
   - Of which 10 did not match due to:
   Overload of function 'mul': File: <numerous>: Line N/A.
     With argument(s): '(int64, list(int64)<iv=None>)':
    No match.
   - Of which 2 did not match due to:
   Operator Overload in function 'mul': File: unknown: Line unknown.
     With argument(s): '(int64, list(int64)<iv=None>)':
    No match for registered cases:
     * (int64, int64) -> int64
     * (int64, uint64) -> int64
     * (uint64, int64) -> int64
     * (uint64, uint64) -> uint64
     * (float32, float32) -> float32
     * (float64, float64) -> float64
     * (complex64, complex64) -> complex64
     * (complex

get symbols for all classes


/home/xxc90/neutronNoise_paper/PFSA.py:626: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "_ravel_index" failed type inference due to: non-precise type pyobject
During: typing of argument at /home/xxc90/neutronNoise_paper/PFSA.py (628)

File "PFSA.py", line 628:
    def _ravel_index(self, x, dims):
        (x_l, x_m, x_n) = x.shape
        ^

  @numba.jit(boundscheck=True)
/home/xxc90/neutronNoise_paper/PFSA.py:626: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "_ravel_index" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "PFSA.py", line 635:
    def _ravel_index(self, x, dims):
        <source elided>
        # output is the index of the hypercube, in the above example case, return 1
        for l in range(0, x_l):
        ^

  @numba.jit(boundscheck=True)
/home/xxc90/.local/lib/python3.8/site-packages/numba/core/

get states for all classes


...

/home/xxc90/anaconda3/envs/python38/lib/python3.8/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:937: LinAlgWarning: Diagonal number 100 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)


class 1
normalization
get states for all classes
get morph matrix
get labels
[0.8542, 0.8803, 0.8863, 0.8886, 0.889, 0.8837, 0.8823, 0.8876, 0.8858]
fs: 1000 Ns: 1000000000 window: 100000 n_partitioning 150
normalization for all classes together
get the same bounds for all classes
get symbols for all classes
get states for all classes
get morph matrix 0.000000
get morph matrix 1.000000
class 0.000000, calculate the left eigenvector corresponding to left eigenvalue 1, this is the state probability vector.
class 0.000000, calculate the right eigenvectors corresponding to right eigenvalue (excludes eigen value 1).
class 1.000000, calculate the left eigenvector corresponding to left eigenvalue 1, this is the state probability vector.
class 1.000000, calculate the right eigenvectors corresponding to right eigenvalue (excludes eigen value 1).
class 0.000000, calculate state weight Chi
class 0.000000, calculate projection matrix 1
class 0.000000, calculate projection matrix 2
class 1.000000, 

/home/xxc90/anaconda3/envs/python38/lib/python3.8/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:937: LinAlgWarning: Diagonal number 150 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)


class 1
normalization
get states for all classes
get morph matrix
get labels
[0.8542, 0.8803, 0.8863, 0.8886, 0.889, 0.8837, 0.8823, 0.8876, 0.8858, 0.8886]
fs: 1000 Ns: 1000000000 window: 100000 n_partitioning 200
normalization for all classes together
get the same bounds for all classes
get symbols for all classes
get states for all classes
get morph matrix 0.000000
get morph matrix 1.000000
class 0.000000, calculate the left eigenvector corresponding to left eigenvalue 1, this is the state probability vector.
class 0.000000, calculate the right eigenvectors corresponding to right eigenvalue (excludes eigen value 1).
class 1.000000, calculate the left eigenvector corresponding to left eigenvalue 1, this is the state probability vector.
class 1.000000, calculate the right eigenvectors corresponding to right eigenvalue (excludes eigen value 1).
class 0.000000, calculate state weight Chi
class 0.000000, calculate projection matrix 1
class 0.000000, calculate projection matrix 2
class 1.

/home/xxc90/anaconda3/envs/python38/lib/python3.8/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:937: LinAlgWarning: Diagonal number 200 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)


### The PC died at the last step. now only the last step will be runned

In [1]:
from scipy.stats import mode
from numpy import linalg as LA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numba
import time
from scipy.sparse.linalg import eigs
from numpy.linalg import inv
import sys
from PFSA import PFSA

In [2]:
acc = np.load("acc_incore_alphabet_sizes.npy")
print(acc)

[0.8542 0.8803 0.8863 0.8886 0.889  0.8837 0.8823 0.8876 0.8858 0.8886
 0.8915 0.8893]


In [3]:
Xn_IC = np.load("X_incore_normal.npz")
Xa_IC = np.load("X_incore_abnormal.npz")

In [4]:
alphabet_sizes = [800] # alphabet sizes
for alphabet_size in alphabet_sizes:
    
    X_class1 = Xn_IC[1:] # class 1 is normal, index is 0
    X_class2 = Xa_IC[1:] # class 2 is abnormal, index is 1
    
    fs = 1000
    Ns = 1000000000
    
    window = int(fs * 100) # 100 seconds
    sample_size = len(X_class1) // window
    X_class1 = X_class1.reshape(sample_size, window)
    X_class1 = X_class1.reshape(*X_class1.shape, 1)
    np.random.shuffle(X_class1)
    Y_class1 = np.zeros(sample_size) # class 1 is labeled by index 0

    X_class2 = X_class2.reshape(sample_size, window)
    X_class2 = X_class2.reshape(*X_class2.shape, 1)
    np.random.shuffle(X_class2)
    Y_class2 = np.ones(sample_size) # class 2 is labled by index 1

    X_train = np.zeros((X_class1.shape[0], X_class1.shape[1], X_class1.shape[2]))
    Y_train = np.zeros(Y_class1.size)

    X_train[0: sample_size // 2] = X_class1[0: sample_size // 2].copy()
    X_train[sample_size // 2 : ] = X_class2[0: sample_size // 2].copy()

    Y_train[0: sample_size // 2] = Y_class1[0: sample_size // 2].copy()
    Y_train[sample_size // 2 : ] = Y_class2[0: sample_size // 2].copy()

    X_test = np.zeros((X_class1.shape[0], X_class1.shape[1], X_class1.shape[2]))
    Y_test = np.zeros(Y_class1.size)

    X_test[0: sample_size // 2] = X_class1[sample_size // 2:].copy()
    X_test[sample_size // 2 : ] = X_class2[sample_size // 2:].copy()

    Y_test[0: sample_size // 2] = Y_class1[sample_size // 2:].copy()
    Y_test[sample_size // 2 : ] = Y_class2[sample_size // 2:].copy()
    
    classfiers = ['projection 2'] # select to only use projection 2 classifier

    n_partitioning = alphabet_size
    depth = 1

    print("fs:", fs, "Ns:", Ns, "window:", window, "n_partitioning", n_partitioning)
    normalizaiton_method = 'mixed classes' # select to use "mixed classes" normalization method
    model = PFSA(n_partitioning = n_partitioning, depth = depth, train_regime= normalizaiton_method)
    model.fit(X_train, Y_train)
    y_test_predicted_raw  = model.predict(X_test, classifier=classfiers)
    y_test_predicted = y_test_predicted_raw
    acc = np.append(acc, sum(y_test_predicted  == Y_test) / len(Y_test))
    print(acc)
    np.save("acc_incore_alphabet_sizes.npy", acc)  # save during running

fs: 1000 Ns: 1000000000 window: 100000 n_partitioning 800
normalization for all classes together
get the same bounds for all classes


/home/xxc90/neutronNoise_paper/PFSA.py:452: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "determine_partitioning_bounds" failed type inference due to: No implementation of function Function(<built-in function mul>) found for signature:
 
 >>> mul(int64, list(int64)<iv=None>)
 
There are 12 candidate implementations:
   - Of which 10 did not match due to:
   Overload of function 'mul': File: <numerous>: Line N/A.
     With argument(s): '(int64, list(int64)<iv=None>)':
    No match.
   - Of which 2 did not match due to:
   Operator Overload in function 'mul': File: unknown: Line unknown.
     With argument(s): '(int64, list(int64)<iv=None>)':
    No match for registered cases:
     * (int64, int64) -> int64
     * (int64, uint64) -> int64
     * (uint64, int64) -> int64
     * (uint64, uint64) -> uint64
     * (float32, float32) -> float32
     * (float64, float64) -> float64
     * (complex64, complex64) -> complex64
     * (complex

get symbols for all classes


/home/xxc90/neutronNoise_paper/PFSA.py:627: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "_ravel_index" failed type inference due to: non-precise type pyobject
During: typing of argument at /home/xxc90/neutronNoise_paper/PFSA.py (629)

File "PFSA.py", line 629:
    def _ravel_index(self, x, dims):
        (x_l, x_m, x_n) = x.shape
        ^

  @numba.jit(boundscheck=True)
/home/xxc90/neutronNoise_paper/PFSA.py:627: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "_ravel_index" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "PFSA.py", line 636:
    def _ravel_index(self, x, dims):
        <source elided>
        # output is the index of the hypercube, in the above example case, return 1
        for l in range(0, x_l):
        ^

  @numba.jit(boundscheck=True)
/home/xxc90/.local/lib/python3.8/site-packages/numba/core/

get states for all classes


...

/home/xxc90/neutronNoise_paper/PFSA.py:573: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "cal_morph_matrix" failed type inference due to: non-precise type pyobject
During: typing of argument at /home/xxc90/neutronNoise_paper/PFSA.py (581)

File "PFSA.py", line 581:
    def cal_morph_matrix(self, x, states_size, regime='MAP'):
        <source elided>
        # print(series_lens)
        x = x.copy()
        ^

  @numba.jit(boundscheck=True)
/home/xxc90/neutronNoise_paper/PFSA.py:573: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "cal_morph_matrix" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "PFSA.py", line 590:
    def cal_morph_matrix(self, x, states_size, regime='MAP'):
        <source elided>

        for m in range(x_m):
        ^

  @numba.jit(boundscheck=True)
/home/xxc90/.local/lib/python3.8/site-packages

get morph matrix 1.000000
class 0.000000, calculate the left eigenvector corresponding to left eigenvalue 1, this is the state probability vector.
class 0.000000, calculate the right eigenvectors corresponding to right eigenvalue (excludes eigen value 1).
class 1.000000, calculate the left eigenvector corresponding to left eigenvalue 1, this is the state probability vector.
class 1.000000, calculate the right eigenvectors corresponding to right eigenvalue (excludes eigen value 1).
class 0.000000, calculate state weight Chi
class 0.000000, calculate projection matrix 1
class 0.000000, calculate projection matrix 2
class 1.000000, calculate state weight Chi
class 1.000000, calculate projection matrix 1
class 1.000000, calculate projection matrix 2
class 0
normalization
get states for all classes
get morph matrix


/home/xxc90/anaconda3/envs/python38/lib/python3.8/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:937: LinAlgWarning: Diagonal number 800 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)


class 1
normalization
get states for all classes
get morph matrix
get labels


AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [5]:
acc = np.append(acc, sum(y_test_predicted  == Y_test) / len(Y_test))

In [6]:
acc

array([0.8542, 0.8803, 0.8863, 0.8886, 0.889 , 0.8837, 0.8823, 0.8876,
       0.8858, 0.8886, 0.8915, 0.8893, 0.8905])

In [7]:
np.save("acc_incore_alphabet_sizes.npy", acc)